In [3]:
import os
import random
from collections import Counter

from PIL import Image
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

In [ ]:
# ----------------------------------------
# 1. Objectenlijst automatisch genereren
# ----------------------------------------
folder_path = (
    "data/preparation/images"  # Pas dit pad aan naar jouw folder met PNG-bestanden
)

# Haal alle PNG-bestanden op in de folder
png_files = [f for f in os.listdir(folder_path) if f.lower().endswith(".png")]

# Extraheer de objectnamen (bestandsnaam zonder extensie)
# (Neem aan dat elk bestand één object representeert)
objects = [os.path.splitext(f)[0] for f in png_files]
# Zorg voor een unieke, gesorteerde lijst
objects = sorted(list(set(objects)))
print("Gevonden objecten:")
for obj in objects:
    print("-", obj)

# ----------------------------------------
# 2. Automatisch selecties genereren
# ----------------------------------------
objects_per_recording = 5


def generate_selections(num_recordings, objects, objects_per_recording, seed):
    random.seed(seed)
    frequency = {obj: 0 for obj in objects}
    selections = []

    for _ in range(num_recordings):
        current_objects = objects.copy()
        random.shuffle(current_objects)
        # Sorteer op basis van de huidige frequentie (laagste eerst)
        current_objects.sort(key=lambda x: frequency[x])
        chosen = current_objects[:objects_per_recording]
        selections.append(chosen)
        for obj in chosen:
            frequency[obj] += 1
    return selections, frequency


# Stel het aantal opnames in (pas dit aan indien nodig)
N = 13
SEED = 42
selections, freq = generate_selections(N, objects, objects_per_recording, SEED)

print("\nGegenereerde selecties per opname:")
for idx, sel in enumerate(selections, start=1):
    print(f"Opname {idx}: {sel}")


# ----------------------------------------
# 3. PDF per selectie genereren
# ----------------------------------------
def generate_pdf_for_selection(selection, output_pdf, folder_path, max_width=200):
    """
    Genereert een PDF voor een enkele selectie (lijst van objectnamen).
    Voor elk object wordt de naam en de bijbehorende PNG-foto (geschaald naar max_width) naast elkaar getekend.
    """
    c = canvas.Canvas(output_pdf, pagesize=letter)
    page_width, page_height = letter

    # Layout parameters
    top_margin = 50
    bottom_margin = 50
    left_margin = 50
    text_x = left_margin
    image_x = left_margin + 150 + 20  # ruimte voor tekst (150) plus tussenruimte (20)
    padding = 10  # extra ruimte per rij

    y = page_height - top_margin  # startpositie van boven

    for object_name in selection:
        # Zoek in de folder naar een PNG-bestand dat de objectnaam bevat (case-insensitive)
        image_file = None
        for f in os.listdir(folder_path):
            if f.lower().endswith(".png") and object_name.lower() in f.lower():
                image_file = f
                break

        if image_file is None:
            print(f"Geen afbeelding gevonden voor '{object_name}'.")
            continue

        image_path = os.path.join(folder_path, image_file)
        # Open de afbeelding en schaal indien nodig (breedte maximaal 200 pixels)
        im = Image.open(image_path)
        orig_width, orig_height = im.size
        if orig_width > max_width:
            scale = max_width / orig_width
            new_width = max_width
            new_height = int(orig_height * scale)
        else:
            new_width = orig_width
            new_height = orig_height

        # Bepaal de hoogte van de rij (minimaal 40 punten voor de tekst plus padding)
        row_height = max(new_height, 40) + padding

        # Als er onvoldoende ruimte is op de huidige pagina, voeg een nieuwe pagina toe
        if y - row_height < bottom_margin:
            c.showPage()
            y = page_height - top_margin

        # Teken de objectnaam (links)
        text_y = y - row_height / 2  # ongeveer verticaal gecentreerd
        c.drawString(text_x, text_y, object_name)

        # Teken de afbeelding (rechts)
        image_y = y - row_height + (padding / 2)
        c.drawImage(
            image_path,
            image_x,
            image_y,
            width=new_width,
            height=new_height,
            preserveAspectRatio=True,
            anchor="sw",
        )

        # Update de y-positie voor de volgende rij
        y -= row_height

    c.save()


# Maak een outputfolder voor de PDF's aan
output_folder = "data/preparation/experimental_setup"
os.makedirs(output_folder, exist_ok=True)

for idx, selection in enumerate(selections, start=1):
    pdf_filename = os.path.join(output_folder, f"opname_{idx}.pdf")
    generate_pdf_for_selection(selection, pdf_filename, folder_path, max_width=200)
    print(f"PDF gegenereerd: {pdf_filename}")

# Toon de uiteindelijke distributie van de objecten
print("\nUiteindelijke distributie van objecten:")
distribution = Counter()
for sel in selections:
    distribution.update(sel)
for obj in sorted(distribution.keys()):
    print(f"{obj}: {distribution[obj]} keer")

Gevonden objecten:
- ampulepoeder
- ampulevloeistof
- bol wol
- bril
- fotokader
- iced tea
- infuus
- keukenmes
- naaldcontainer
- nuchter
- rollator
- monitor
- snoep
- spuit
- stethoscoop

Gegenereerde selecties per opname:
Opname 1: ['naaldcontainer', 'spuit', 'keukenmes', 'infuus', 'stethoscoop']
Opname 2: ['bol wol', 'snoep', 'nuchter', 'fotokader', 'iced tea']
Opname 3: ['bril', 'monitor', 'rollator', 'ampulevloeistof', 'ampulepoeder']
Opname 4: ['snoep', 'naaldcontainer', 'infuus', 'bol wol', 'rollator']
Opname 5: ['iced tea', 'stethoscoop', 'keukenmes', 'fotokader', 'spuit']
Opname 6: ['monitor', 'ampulevloeistof', 'ampulepoeder', 'nuchter', 'bril']
Opname 7: ['nuchter', 'stethoscoop', 'monitor', 'spuit', 'ampulevloeistof']
Opname 8: ['naaldcontainer', 'ampulepoeder', 'bol wol', 'rollator', 'infuus']
Opname 9: ['keukenmes', 'bril', 'snoep', 'iced tea', 'fotokader']
Opname 10: ['naaldcontainer', 'ampulepoeder', 'iced tea', 'monitor', 'keukenmes']
Opname 11: ['infuus', 'nuchter'